In [ ]:
'''
added perturbation for features, earlier it was only for centroids

training trained model from code - ch2, starting from 38th epoch to do adversarial training with FGSM and later with PGD

model saved as: adv+QNI_model.pth 
pgd+fgsm model: adv+QNI+PGD_model.pth


'''

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms
from collections import Counter
import numpy as np
import random
import os
from torchvision.datasets import ImageFolder
from matplotlib import pyplot as plt
import pennylane as qml
from pennylane.qnn import TorchLayer
from tqdm.notebook import tqdm

#for loss function 
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Set seeds for reproducibility
def seed_all(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_all(42)

# ========== DEVICE ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== PARAMETERS ==========
n_qubits = 6
batch_size = 16
num_classes = 25
num_epochs = 50
lr = 0.0005

# ========== TRANSFORMS WITH DATA AUGMENTATION ==========
# ✅ For training (with augmentation)
train_transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ✅ For validation and test (no augmentation)
eval_transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# ========== DATASETS ==========
train_dataset = ImageFolder('/home/netsec1/dataset_folder/malimg_dataset/train', transform=train_transform)
val_dataset   = ImageFolder('/home/netsec1/dataset_folder/malimg_dataset/val', transform=eval_transform)
test_dataset  = ImageFolder('/home/netsec1/dataset_folder/malimg_dataset/test', transform=eval_transform)
print("**dataset loaded**")
# ========== CLASS WEIGHTS ==========
from sklearn.utils.class_weight import compute_class_weight

labels = [label for _, label in train_dataset.samples]
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels),
                                     y=labels)
class_wts = torch.tensor(class_weights, dtype=torch.float)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ========== QUANTUM CIRCUIT ==========
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")

def quantum_circuit(inputs, weights):
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    
    for l in range(weights.shape[0]):
        for i in range(n_qubits):
            qml.RY(weights[l][i], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i+1])
    
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (6, n_qubits)}


# ========== CNN + QNN MODEL ==========
class FeatureReduce(nn.Module):
    def __init__(self, final_dim, dropout=0.4):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),    # 128 -> 64
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Conv2d(8, 16, 3, stride=2, padding=1),   # 64 -> 32
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # 32 -> 16
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # 16 -> 8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Conv2d(64, 128, 3, stride=2, padding=1),  # ⬅️ Extra block: 8 -> 4
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))                # 4×4 -> 1×1
        )
        self.fc = nn.Linear(128, final_dim)  # ⬅️ Changed from 64 to 128

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class HybridQNN(nn.Module):
    def __init__(self, n_qubits, num_classes):
        super().__init__()
        self.feature_extractor = FeatureReduce(final_dim=n_qubits)
        self.q_layer = TorchLayer(quantum_circuit, weight_shapes)

        # Adding 4-layer MLP after quantum layer
        self.classifier = nn.Sequential(
            nn.Linear(n_qubits, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.tanh(x)
        q_out = torch.stack([self.q_layer(f) for f in x])
        return self.classifier(q_out)

# ========== TRAINING ==========
print("Starting training")

# Applying gradient based pertubations of features
def gradient_noise_on_features(model, x, y, epsilon=0.1):
    """
    Compute gradient of loss w.r.t. extracted features and perturb them.
    Returns: perturbed feature tensor [B, n_qubits]
    """
    model.eval()
    x = x.clone().detach().requires_grad_(True)
    
    # Forward: extract features, apply tanh, quantum, classify
    feats = model.feature_extractor(x)  # pre-tanh features [B, n_qubits]
    feats_tanh = torch.tanh(feats)
    
    q_out = torch.stack([model.q_layer(f) for f in feats_tanh])
    logits = model.classifier(q_out)
    
    loss = F.cross_entropy(logits, y)
    loss.backward()
    
    # Get gradient w.r.t. input features
    feats_grad = x.grad.data
    x_pert = x + epsilon * feats_grad.sign()
    x_pert = torch.clamp(x_pert, -1, 1)
    
    # Re-extract features from perturbed image
    feats_pert = model.feature_extractor(x_pert)
    
    return feats_pert.detach()

# ── 2) Precompute class‐centroids in feature space ──────────────────────────
def compute_centroids(model, loader, device, num_classes):
    model.eval()
    sums = torch.zeros(num_classes, n_qubits, device=device)
    counts = torch.zeros(num_classes, device=device)
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            feats = model.feature_extractor(x)      # pre‐tanh features
            for c in range(num_classes):
                mask = (y==c)
                if mask.any():
                    sums[c] += feats[mask].sum(0)
                    counts[c] += mask.sum()
    return sums / counts.unsqueeze(1)

# ── 3) QNI perturbation function ────────────────────────────────────────────
def gradient_based_noise(model, x, y, epsilon=0.1):
    """
    x: input image batch [B, C, H, W]
    y: labels
    """
    x = x.clone().detach().requires_grad_(True)
    model.eval()

    # Get output logits
    logits = model(x)
    loss = F.cross_entropy(logits, y)

    # Compute gradient of loss w.r.t input
    loss.backward()
    grad = x.grad.data  # [B, C, H, W]

    # Normalize gradient and perturb input
    grad_sign = grad.sign()
    x_pert = x + epsilon * grad_sign
    x_pert = torch.clamp(x_pert, -1, 1)  # Keep within normalized bounds

    return x_pert.detach()

# … everything above stays the same up to compute_centroids …

# ── 4) Training loop with QNI ──────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridQNN(n_qubits, num_classes).to(device)
opt   = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=5e-3)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=5)

# Initialize best validation accuracy
best_val_acc = 0.0
best_model_path = "best_QNI_model_2.pth"


# helper to evaluate on a loader
def evaluate(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(1)
            correct += (preds == y).sum().item()
            total   += y.size(0)
    return correct/total

# initial centroids before training
centroids = compute_centroids(model, train_loader, device, num_classes)

# for epoch in range(1, 51):
#     # every 5 epochs, recompute centroids on the *current* model:
#     if epoch % 5 == 0:
#         centroids = compute_centroids(model, train_loader, device, num_classes)
    
#     model.train()
#     running_loss, running_correct, running_total = 0, 0, 0

#     for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [train]"):
#         x, y = x.to(device), y.to(device)

#         ### changed
#         # Clean path
#         feats = model.feature_extractor(x)
#         clean_logits = model(x)
#         loss_clean = F.cross_entropy(clean_logits, y)

#         # Perturbed path (gradient-based on features)
#         feats_pert = gradient_noise_on_features(model, x, y, epsilon=0.1)
#         feats_pert_t = torch.tanh(feats_pert)
#         q_out_pert = torch.stack([model.q_layer(f) for f in feats_pert_t])
#         pert_logits = model.classifier(q_out_pert)
#         loss_pert = F.cross_entropy(pert_logits, y)

#         # Joint loss
#         loss = 0.8 * loss_clean + 0.2 * loss_pert
#         opt.zero_grad()
#         loss.backward()
#         opt.step()


#         # track
#         running_loss   += loss.item() * x.size(0)
#         running_correct += (clean_logits.argmax(1) == y).sum().item()
#         running_total   += y.size(0)

#     # step scheduler on *average* training loss
#     avg_train_loss = running_loss / running_total
#     sched.step(avg_train_loss)

#     train_acc = running_correct / running_total
#     val_acc   = evaluate(model, val_loader)
#     print(f"\nEpoch {epoch:2d} — train loss: {avg_train_loss:.4f}, "
#       f"train acc: {train_acc:.4f}, val acc: {val_acc:.4f}")
    
#     # Save best model
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(), 
#             'optimizer_state_dict': opt.state_dict(),
#             'val_accuracy': val_acc
#         }, best_model_path)F
#         print(f"✅ Best model saved at epoch {epoch} with val_acc: {val_acc:.4f}\n")
#     else:
#         print()

**dataset loaded**
Starting training


In [7]:
### for adversarial training

import torch
import torch.nn.functional as F
from tqdm import tqdm

# Load previous model
model = HybridQNN(n_qubits, num_classes).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=5e-3)

# Load checkpoint
checkpoint = torch.load("best_QNI_model_2.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
best_val_acc = checkpoint['val_accuracy']
print(f"✅ Model loaded. Resuming from epoch {start_epoch} with best val_acc = {best_val_acc:.4f}")

# FGSM adversarial example generator
def fgsm_attack_inputs(model, x, y, epsilon=0.1):
    x_adv = x.clone().detach().requires_grad_(True).to(device)
    y = y.to(device)
    model.eval()
    logits = model(x_adv)
    loss = F.cross_entropy(logits, y)
    loss.backward()
    x_adv = x_adv + epsilon * x_adv.grad.sign()
    return torch.clamp(x_adv, -1.0, 1.0).detach()

# Resume training for 10 more epochs with adversarial examples
for epoch in range(start_epoch, start_epoch + 10):
    if epoch % 5 == 0:
        centroids = compute_centroids(model, train_loader, device, num_classes)

    model.train()
    running_loss, running_correct, running_total = 0, 0, 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [adv-train]"):
        x, y = x.to(device), y.to(device)

        # Clean input loss
        clean_logits = model(x)
        loss_clean = F.cross_entropy(clean_logits, y)

        # Adversarial input loss (FGSM)
        x_adv = fgsm_attack_inputs(model, x, y, epsilon=0.1)
        adv_logits = model(x_adv)
        loss_adv = F.cross_entropy(adv_logits, y)

        # Combine losses
        loss = 0.7 * loss_clean + 0.3 * loss_adv
        opt.zero_grad()
        loss.backward()
        opt.step()

        # Track
        running_loss += loss.item() * x.size(0)
        running_correct += (clean_logits.argmax(1) == y).sum().item()
        running_total += y.size(0)

    avg_train_loss = running_loss / running_total
    sched.step(avg_train_loss)

    train_acc = running_correct / running_total
    val_acc = evaluate(model, val_loader)
    print(f"\nEpoch {epoch:2d} — train loss: {avg_train_loss:.4f}, "
          f"train acc: {train_acc:.4f}, val acc: {val_acc:.4f}")

   # Save only the best model during adversarial training
   
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'val_accuracy': val_acc
        }, "adv+QNI_model.pth")
    print(f"✅ Best adversarial model saved at epoch {epoch} with val_acc: {val_acc:.4f}\n")


✅ Model loaded. Resuming from epoch 37 with best val_acc = 0.9697


Epoch 37 [adv-train]: 100%|██████████| 467/467 [16:20<00:00,  2.10s/it]



Epoch 37 — train loss: 0.4874, train acc: 0.9596, val acc: 0.9621
✅ Best adversarial model saved at epoch 37 with val_acc: 0.9621



Epoch 38 [adv-train]: 100%|██████████| 467/467 [16:30<00:00,  2.12s/it]



Epoch 38 — train loss: 0.3516, train acc: 0.9630, val acc: 0.9502
✅ Best adversarial model saved at epoch 38 with val_acc: 0.9502



...


Epoch 44 — train loss: 0.2124, train acc: 0.9688, val acc: 0.9642
✅ Best adversarial model saved at epoch 44 with val_acc: 0.9642



Epoch 45 [adv-train]: 100%|██████████| 467/467 [15:44<00:00,  2.02s/it]



Epoch 45 — train loss: 0.1978, train acc: 0.9681, val acc: 0.9686
✅ Best adversarial model saved at epoch 45 with val_acc: 0.9686



Epoch 46 [adv-train]: 100%|██████████| 467/467 [16:20<00:00,  2.10s/it]



Epoch 46 — train loss: 0.1930, train acc: 0.9674, val acc: 0.9664
✅ Best adversarial model saved at epoch 46 with val_acc: 0.9664



In [4]:
# example: adv+QNI_model.pth (from FGSM training)
checkpoint = torch.load("adv+QNI_model.pth", map_location=device)

model = HybridQNN(n_qubits, num_classes).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=5e-3)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=5)

model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
best_val_acc = checkpoint['val_accuracy']
print(f"✅ Resuming from FGSM-trained model | Epoch {start_epoch}, Best Val Acc: {best_val_acc:.4f}")

def pgd_attack_inputs(model, x, y, eps=0.1, alpha=0.01, iters=7):
    x_adv = x.clone().detach().to(device)
    x_adv = x_adv + torch.empty_like(x_adv).uniform_(-eps, eps)
    x_adv = torch.clamp(x_adv, -1.0, 1.0).detach()

    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = F.cross_entropy(logits, y)
        model.zero_grad()
        loss.backward()
        x_adv = x_adv + alpha * x_adv.grad.sign()
        delta = torch.clamp(x_adv - x, -eps, eps)
        x_adv = torch.clamp(x + delta, -1.0, 1.0).detach()
    return x_adv

# Start PGD training from epoch 47
# for epoch in range(start_epoch, start_epoch + 10):  # you can increase 10 to more if needed
#     if epoch % 5 == 0:
#         centroids = compute_centroids(model, train_loader, device, num_classes)

#     model.train()
#     running_loss, running_correct, running_total = 0, 0, 0

#     for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [PGD-adv-train]"):
#         x, y = x.to(device), y.to(device)

#         # Clean input loss
#         clean_logits = model(x)
#         loss_clean = F.cross_entropy(clean_logits, y)

#         # PGD adversarial input
#         x_pgd = pgd_attack_inputs(model, x, y, eps=0.1, alpha=0.01, iters=7)
#         logits_pgd = model(x_pgd)
#         loss_pgd = F.cross_entropy(logits_pgd, y)

#         # Combine losses
#         loss = 0.6 * loss_clean + 0.4 * loss_pgd

#         opt.zero_grad()
#         loss.backward()
#         opt.step()

#         running_loss += loss.item() * x.size(0)
#         running_correct += (clean_logits.argmax(1) == y).sum().item()
#         running_total += x.size(0)

#     avg_train_loss = running_loss / running_total
#     train_acc = running_correct / running_total
#     sched.step(avg_train_loss)

#     val_acc = evaluate(model, val_loader)

#     print(f"\nEpoch {epoch:2d} — train loss: {avg_train_loss:.4f}, "
#           f"train acc: {train_acc:.4f}, val acc: {val_acc:.4f}")

    
#     best_val_acc = val_acc
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': opt.state_dict(),
#             'val_accuracy': val_acc
#         }, "adv+QNI+PGD_model.pth")
#     print(f"✅ PGD adversarial model saved at epoch {epoch} with val_acc: {val_acc:.4f}\n")


✅ Resuming from FGSM-trained model | Epoch 47, Best Val Acc: 0.9664


In [6]:
# === Load the PGD + QNI checkpoint for continuing training on PGD ===
checkpoint_path = "adv+QNI+PGD_model.pth"

checkpoint = torch.load(checkpoint_path, map_location=device)

model = HybridQNN(n_qubits, num_classes).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=5e-3)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=5)

model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

start_epoch = checkpoint['epoch'] + 1
best_val_acc = checkpoint['val_accuracy']

print(f"🔄 Resuming training from epoch {start_epoch}, best val acc so far: {best_val_acc:.4f}")

# === Continue training ===
for epoch in range(start_epoch, start_epoch + 10):  # ⬅️ change to how many more epochs you want
    if epoch % 5 == 0:
        centroids = compute_centroids(model, train_loader, device, num_classes)

    model.train()
    running_loss, running_correct, running_total = 0, 0, 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [Continue PGD Adv Train]"):
        x, y = x.to(device), y.to(device)

        # Clean loss
        logits_clean = model(x)
        loss_clean = F.cross_entropy(logits_clean, y)

        # PGD adversarial input
        x_pgd = pgd_attack_inputs(model, x, y, eps=0.1, alpha=0.01, iters=7)
        logits_pgd = model(x_pgd)
        loss_pgd = F.cross_entropy(logits_pgd, y)

        # Combined loss
        loss = 0.6 * loss_clean + 0.4 * loss_pgd

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item() * x.size(0)
        running_correct += (logits_clean.argmax(1) == y).sum().item()
        running_total += x.size(0)

    train_loss = running_loss / running_total
    train_acc = running_correct / running_total
    sched.step(train_loss)

    val_acc = evaluate(model, val_loader)

    print(f"📊 Epoch {epoch:2d} | Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Save best or every checkpoint
    best_val_acc = max(best_val_acc, val_acc)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
        'val_accuracy': best_val_acc
    }, "adv+QNI+PGD_model.pth")
    print(f"💾 Model saved after epoch {epoch} with val acc: {val_acc:.4f}\n")


🔄 Resuming training from epoch 62, best val acc so far: 0.9664


Epoch 62 [Continue PGD Adv Train]:   0%|          | 0/467 [00:00<?, ?it/s]

📊 Epoch 62 | Train Loss: 0.4012 | Train Acc: 0.9021 | Val Acc: 0.9697
💾 Model saved after epoch 62 with val acc: 0.9697



Epoch 63 [Continue PGD Adv Train]:   0%|          | 0/467 [00:00<?, ?it/s]

📊 Epoch 63 | Train Loss: 0.3982 | Train Acc: 0.9000 | Val Acc: 0.9198
💾 Model saved after epoch 63 with val acc: 0.9198



Epoch 64 [Continue PGD Adv Train]:   0%|          | 0/467 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
## to evaluate the saved model

# === Evaluate saved model ===
model = HybridQNN(n_qubits, num_classes).to(device)
checkpoint = torch.load("adv+QNI_model.pth", map_location=device)
model.load_state_dict(checkpoint['model_s tate_dict'])
model.eval()

def evaluate(model, loader):
    total, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

val_acc = evaluate(model, val_loader)
test_acc = evaluate(model, test_loader)

print(f"✅ Validation Accuracy: {val_acc:.4f}")
print(f"✅ Test Accuracy: {test_acc:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'adv+QNI_model.pth'

In [ ]:
from sklearn.metrics import classification_report
import torch

# Initialize the model
model = HybridQNN(n_qubits=n_qubits, num_classes=num_classes).to(device)

# ✅ Load only the model weights from the saved checkpoint
checkpoint = torch.load("best_QNI_model_2.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])  # ⬅️ fix
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Get class names from the test dataset
class_names = test_dataset.classes

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))


In [8]:
## performing FGSM attack

import torch
import torch.nn.functional as F

# ===== FGSM Attack =====
def fgsm_attack(model, x, y, epsilon=0.1, device='cuda'):
    model.eval()
    x_adv = x.clone().detach().to(device).requires_grad_(True)
    y = y.to(device)

    logits = model(x_adv)
    loss = F.cross_entropy(logits, y)
    model.zero_grad()
    loss.backward()

    x_adv = x_adv + epsilon * x_adv.grad.sign()
    x_adv = torch.clamp(x_adv, min=-1.0, max=1.0)
    return x_adv.detach()

# ===== PGD Attack =====
def pgd_attack(model, x, y, eps=0.1, alpha=0.02, iters=10, device='cuda'):
    model.eval()
    x_orig = x.clone().detach().to(device)
    x_adv  = x_orig + torch.empty_like(x_orig).uniform_(-eps, eps)
    x_adv  = torch.clamp(x_adv, -1.0, 1.0).detach()

    y = y.to(device)
    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = F.cross_entropy(logits, y)
        model.zero_grad()
        loss.backward()

        x_adv = x_adv + alpha * x_adv.grad.sign()
        delta = torch.clamp(x_adv - x_orig, min=-eps, max=eps)
        x_adv = torch.clamp(x_orig + delta, -1.0, 1.0).detach()
    return x_adv

# ===== Evaluate Attack =====
def test_adversarial(model, loader, attack_fn, attack_name, **attack_kwargs):
    model.eval()
    total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        x_adv = attack_fn(model, x, y, **attack_kwargs)
        logits = model(x_adv)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    acc = 100. * correct / total
    print(f"{attack_name} Adversarial Accuracy: {acc:.2f}%")

# ===== Load Model from best_QNI_model_2.pth =====
model = HybridQNN(n_qubits=n_qubits, num_classes=num_classes).to(device)
checkpoint = torch.load("adv+QNI_model.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# ===== Run Evaluations =====
# Clean accuracy
clean_acc = evaluate(model, test_loader)
print(f"\n✅ Clean Test Accuracy: {clean_acc*100:.2f}%")

# FGSM attack
test_adversarial(
    model, test_loader,
    attack_fn=fgsm_attack,
    attack_name="FGSM",
    epsilon=0.1,
    device=device
)

# PGD attack
test_adversarial(
    model, test_loader,
    attack_fn=pgd_attack,
    attack_name="PGD",
    eps=0.1,
    alpha=0.02,
    iters=10,
    device=device
)



✅ Clean Test Accuracy: 95.63%
FGSM Adversarial Accuracy: 83.14%
PGD Adversarial Accuracy: 23.20%


In [ ]:
# ===== claude FGSM, PGD Attack code =====
def fgsm_attack(model, x, y, epsilon=0.1, device='cuda'):
    """
    Fast Gradient Sign Method (FGSM) attack
    
    Args:
        model: The target model
        x: Input tensor
        y: True labels
        epsilon: Attack strength
        device: Device to run on
    
    Returns:
        Adversarial examples
    """
    model.eval()
    x_adv = x.clone().detach().to(device).requires_grad_(True)
    y = y.to(device)
    
    # Forward pass
    logits = model(x_adv)
    loss = F.cross_entropy(logits, y)
    
    # Backward pass
    model.zero_grad()
    loss.backward()
    
    # Generate adversarial example
    x_adv = x_adv + epsilon * x_adv.grad.sign()
    x_adv = torch.clamp(x_adv, min=-1.0, max=1.0)
    
    return x_adv.detach()


# ===== IMPROVED PGD Attack =====
def pgd_attack(model, x, y, eps=0.1, alpha=0.02, iters=10, device='cuda', random_start=True):
    """
    Projected Gradient Descent (PGD) attack
    
    Args:
        model: The target model
        x: Input tensor
        y: True labels
        eps: Maximum perturbation (L∞ norm)
        alpha: Step size
        iters: Number of iterations
        device: Device to run on
        random_start: Whether to start from random point in eps-ball
    
    Returns:
        Adversarial examples
    """
    model.eval()
    x_orig = x.clone().detach().to(device)
    y = y.to(device)
    
    # Initialize adversarial example
    if random_start:
        x_adv = x_orig + torch.empty_like(x_orig).uniform_(-eps, eps)
        x_adv = torch.clamp(x_adv, -1.0, 1.0).detach()
    else:
        x_adv = x_orig.clone().detach()
    
    # PGD iterations
    for i in range(iters):
        x_adv.requires_grad_(True)
        
        # Forward pass
        logits = model(x_adv)
        loss = F.cross_entropy(logits, y)
        
        # Backward pass
        model.zero_grad()
        loss.backward()
        
        # Update adversarial example
        x_adv = x_adv + alpha * x_adv.grad.sign()
        
        # Project back to eps-ball around original input
        delta = torch.clamp(x_adv - x_orig, min=-eps, max=eps)
        x_adv = torch.clamp(x_orig + delta, -1.0, 1.0).detach()
    
    return x_adv


# ===== ADDITIONAL ATTACK METHODS =====

def c_w_attack(model, x, y, c=1.0, kappa=0, max_iter=1000, learning_rate=0.01, device='cuda'):
    """
    Carlini & Wagner (C&W) L2 attack
    
    Args:
        model: The target model
        x: Input tensor
        y: True labels
        c: Confidence parameter
        kappa: Confidence margin
        max_iter: Maximum iterations
        learning_rate: Learning rate for optimization
        device: Device to run on
    
    Returns:
        Adversarial examples
    """
    model.eval()
    x_orig = x.clone().detach().to(device)
    y = y.to(device)
    
    # Initialize perturbation parameter
    w = torch.zeros_like(x_orig, requires_grad=True, device=device)
    optimizer = torch.optim.Adam([w], lr=learning_rate)
    
    best_adv = x_orig.clone()
    best_l2 = float('inf')
    
    for i in range(max_iter):
        # Generate adversarial example using tanh transformation
        x_adv = 0.5 * (torch.tanh(w) + 1.0)
        
        # Forward pass
        logits = model(x_adv)
        
        # C&W loss function
        # f(x) = max(max{Z_i: i != t} - Z_t, -kappa)
        real_logits = logits.gather(1, y.unsqueeze(1)).squeeze(1)
        other_logits = logits.clone()
        other_logits.scatter_(1, y.unsqueeze(1), -float('inf'))
        max_other_logits = other_logits.max(1)[0]
        
        f_loss = torch.clamp(max_other_logits - real_logits + kappa, min=0)
        
        # L2 distance
        l2_loss = torch.norm(x_adv - x_orig, p=2, dim=(1, 2, 3))
        
        # Total loss
        loss = l2_loss + c * f_loss
        
        optimizer.zero_grad()
        loss.mean().backward()
        optimizer.step()
        
        # Update best adversarial example
        for j in range(x_orig.size(0)):
            if f_loss[j] < 1e-4 and l2_loss[j] < best_l2:
                best_l2 = l2_loss[j].item()
                best_adv[j] = x_adv[j].detach()
        
        if i % 100 == 0:
            print(f"Iteration {i}: L2 loss = {l2_loss.mean().item():.4f}, "
                  f"F loss = {f_loss.mean().item():.4f}")
    
    return best_adv


def evaluate_robustness(model, test_loader, attacks, device='cuda'):
    """
    Evaluate model robustness against multiple attacks
    
    Args:
        model: Model to evaluate
        test_loader: Test data loader
        attacks: Dictionary of attack functions
        device: Device to run on
    
    Returns:
        Dictionary of accuracy results
    """
    model.eval()
    results = {}
    
    # Clean accuracy
    clean_correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            pred = logits.argmax(1)
            clean_correct += (pred == y).sum().item()
            total += y.size(0)
    
    results['clean'] = clean_correct / total
    print(f"Clean accuracy: {results['clean']:.4f}")
    
    # Adversarial accuracy for each attack
    for attack_name, attack_func in attacks.items():
        adv_correct = 0
        total = 0
        
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            
            # Generate adversarial examples
            x_adv = attack_func(model, x, y, device=device)
            
            # Evaluate on adversarial examples
            with torch.no_grad():
                logits = model(x_adv)
                pred = logits.argmax(1)
                adv_correct += (pred == y).sum().item()
                total += y.size(0)
        
        results[attack_name] = adv_correct / total
        print(f"{attack_name} accuracy: {results[attack_name]:.4f}")
    
    return results


# ===== USAGE EXAMPLE =====
def test_attacks():
    """
    Example usage of the attack functions
    """
    # Define attacks
    attacks = {
        'fgsm_0.1': lambda model, x, y, device: fgsm_attack(model, x, y, epsilon=0.1, device=device),
        'fgsm_0.2': lambda model, x, y, device: fgsm_attack(model, x, y, epsilon=0.2, device=device),
        'pgd_0.1': lambda model, x, y, device: pgd_attack(model, x, y, eps=0.1, alpha=0.02, iters=10, device=device),
        'pgd_0.2': lambda model, x, y, device: pgd_attack(model, x, y, eps=0.2, alpha=0.04, iters=20, device=device),
    }
    
    # Evaluate robustness (assuming model and test_loader are defined)
    # results = evaluate_robustness(model, test_loader, attacks, device='cuda')
    
    return attacks